<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/3-LangChain/langchain_retrieval_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1><a href="https://github.com/peremartra/Large-Language-Model-Notebooks-Course">LLM Hands On Course</a></h1>
    <h3>Understand And Apply Large Language Models</h3>
    <p>by <b>Pere Martra</b></p>
    <h2>Create a Medical Assistant RAG System chat with LangChain & ChromaDB</h2>
</div>


<div>
    &nbsp;
    <a target="_blank" href="https://www.linkedin.com/in/pere-martra/"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>
    
</div>


#Installing libraries & Loading Dataset

In [1]:
!pip install -q openai==1.10.0
!pip install -q langchain==0.1.4
!pip install -q langchain-openai==0.0.5
!pip install --upgrade -q tiktoken==0.5.2
!pip install --upgrade -q datasets==2.16.1
!pip install --upgrade -q chromadb==0.4.22

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

We will download the dataset from the Hugging Face datasets library. It's a dataset with information about diseases.

In [6]:
from datasets import load_dataset

data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
data = data.to_pandas()
data.head(10)

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
5,prevention,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...
6,information,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...
7,susceptibility,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
8,exams and tests,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ..."
9,treatment,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...


In [40]:
data[70:100]

,qtype,Question,Answer
70,prevention,How to prevent Parasites - Baylisascaris infec...,Baylisascaris infection can be prevented by av...
71,information,What is (are) Parasites - Babesiosis ?,Babesiosis is caused by microscopic parasites ...
72,susceptibility,Who is at risk for Parasites - Babesiosis? ?,People can get infected with Babesia parasites...
73,exams and tests,How to diagnose Parasites - Babesiosis ?,"In symptomatic people, babesiosis usually is d..."
74,treatment,What are the treatments for Parasites - Babesi...,Effective treatments are available. People who...
75,prevention,How to prevent Parasites - Babesiosis ?,Steps can be taken to reduce the risk for babe...
76,symptoms,what are the signs and symptoms of rabies?,The first symptoms of rabies may be very simil...
77,susceptibility,what is the risk for my pet for Rabies ?,Any animal bitten or scratched by either a wil...
78,exams and tests,how is rabies diagnosed?,"In animals, rabies is diagnosed using the dire..."
79,information,what is botulism?,Botulism is a rare but serious paralytic illne...


As you can see, the medical information in the dataset is well-organized, and to someone like me, who is not an expert in the field, it appears to be quite valuable. This information could be a useful addition to any general medicine book to support primary care doctors.

Load the langchain libraries to load the document.

In [9]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

The Document is in the Answer column, and the others columns are Metadata.

In [10]:
df_loader = DataFrameLoader(data, page_content_column="Answer")


In [11]:
df_document = df_loader.load()
display(df_document[:2])

[Document(page_content='LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.', metadata={'qtype': 'susceptibility', 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?'}),
 Document(page_content='LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. \n                \nFor infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial 

We can chunk the documents. The size to which we want to split the document is a design decision. The larger it is, the larger the prompt will be, and the slower the Model's response process.

We also need to consider the maximum prompt size and ensure that the document does not exceed it.

In [12]:
from langchain.text_splitter import CharacterTextSplitter

In [13]:
text_splitter = CharacterTextSplitter(chunk_size=1250, chunk_overlap=100)
texts = text_splitter.split_documents(df_document)


These warnings we see are because it can't perform the partition of the required size. This is because it waits for a page break to divide the text and does so when possible.

In [14]:
first_doc = texts[1]
print(first_doc.page_content)

LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. 
                
For infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial phase, which may last as long as a week, typically begins with any or all of the following symptoms: fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. Other symptoms appearing less frequently include sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain. 
                
Following a few days of recovery, a second phase of illness may occur. Symptoms may consist of meningitis (fever, headache, stiff neck, etc.), encephalitis (drowsiness, confusion, sensory disturbances, and/or motor abnormalities, such as paralysis), or meningoencephalitis (inflammation 

### Initialize the Embedding Model and Vector DB

We load the text-embedding-ada-002 model from OpenAI.

In [19]:
from getpass import getpass
OPENAI_API_KEY = getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [21]:

from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'
#model_name = 'text-embedding-3-small'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

The execution of this cell may take 3 to 5 minutes. If you want it to be faster, you can reduce the number of records in the dataset.

In [22]:
directory_cdb = '/content/drive/MyDrive/chromadb'
chroma_db = Chroma.from_documents(
    df_document, embed, persist_directory=directory_cdb
)

We are going to create three objects.

* The language model, which can be any of those from OpenAI, the most common being gpt-3.5.
* The memory, responsible for keeping the prompt with all the necessary history.
* The retrieval, used to obtain information stored in ChromaDB.

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages srtored in memory
    return_messages=True #Must return the messages in the response.
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_db.as_retriever()
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


We can try the isolated Retrieval to see if the information it returns is relevant.




In [24]:
qa.run("What is the main symptom of LCM?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'The main symptom of LCM (lymphocytic choriomeningitis) can vary depending on the phase of the illness. In the initial phase, common symptoms include fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. In the second phase, symptoms may consist of meningitis (fever, headache, stiff neck), encephalitis (drowsiness, confusion, sensory disturbances, paralysis), or meningoencephalitis (inflammation of the brain and meninges). Other symptoms that may occur less frequently include sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain.'

Perfect! The information returned is exactly what we desired.

## Creating the Agent.

In [25]:
from langchain.agents import Tool

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Medical KB',
        func=qa.run,
        description=(
            'use this tool when answering medical knowledge queries to get '
            'more information about the topic'
        )
    )
]

In [26]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


### Using the Conversational Agent

To make queries we simply call the `agent` directly.

First i will try a order not related to the Medical field.

In [27]:
agent("Give me the area of square of 2x2")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "The area of a square with side length 2 is 4 square units."
}

> Finished chain.


{'input': 'Give me the area of square of 2x2',
 'chat_history': [],
 'output': 'The area of a square with side length 2 is 4 square units.'}

Perfect, the model has responded without accessing the configured knowledge database.

Now I will try with a question that is also not related to health.

In [28]:
agent("Do you know the secret identity of SuperMan?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Superman's secret identity is Clark Kent."
}

> Finished chain.


{'input': 'Do you know the secret identity of SuperMan?',
 'chat_history': [HumanMessage(content='Give me the area of square of 2x2'),
  AIMessage(content='The area of a square with side length 2 is 4 square units.')],
 'output': "Superman's secret identity is Clark Kent."}

It has not accessed either, as the model has been able to identify that it is not a question related to the database that LangChain provides.

Now it's time to try with a question related to Medicine. Let's see if the model can understand that it should first look for information in the vector database at its disposal.

In [43]:
 agent.memory.clear()

In [46]:
agent("""I have a patient that can have Botulism,
how can I confirm the diagnose?""")



> Entering new AgentExecutor chain...
{
    "action": "Medical KB",
    "action_input": "diagnosis of botulism"
}
Observation: Physicians may consider the diagnosis of botulism if the patient's history and physical examination suggest it. However, these clues are usually not enough to confirm a diagnosis of botulism. Other conditions such as Guillain-Barré syndrome, stroke, and myasthenia gravis can have similar symptoms, so special tests may be needed to rule out these other conditions. These tests may include a brain scan, spinal fluid examination, nerve conduction test, and a tensilon test for myasthenia gravis. Tests for botulinum toxin and the bacteria that cause botulism can be performed at certain laboratories.
Thought:{
    "action": "Final Answer",
    "action_input": "To confirm a diagnosis of botulism, physicians may consider the patient's history and physical examination. However, these clues are usually not enough to confirm the diagnosis. Special tests may be needed to 

{'input': 'I have a patient that can have Botulism, \nI need to confirm it ASAP what is the first diagnose I can do?',
 'chat_history': [HumanMessage(content='I have a patient that can have Botulism, \nhow can I confirm the diagnose?'),
  AIMessage(content="To confirm a diagnosis of botulism, physicians may consider the patient's history and physical examination. However, these clues are usually not enough to confirm the diagnosis. Special tests may be needed to rule out other conditions with similar symptoms, such as Guillain-Barré syndrome, stroke, and myasthenia gravis. These tests may include a brain scan, spinal fluid examination, nerve conduction test, and a tensilon test for myasthenia gravis. Additionally, tests for botulinum toxin and the bacteria that cause botulism can be performed at certain laboratories."),
  HumanMessage(content='Is this a serious illness?'),
  AIMessage(content='Yes, botulism is a serious illness that can be fatal if not promptly treated.')],
 'output': 

Perfect, the most important thing for us is that it has been able to identify that it should go to the medical database to search for information about the symptoms.

In [45]:
agent("Is this a serious illness?")



> Entering new AgentExecutor chain...
{
    "action": "Medical KB",
    "action_input": "Botulism"
}
Observation: Botulism is a rare but serious illness caused by a toxin produced by the bacterium Clostridium botulinum. There are several types of botulism, including foodborne botulism, wound botulism, infant botulism, adult intestinal toxemia botulism, and iatrogenic botulism. All forms of botulism can be fatal and are considered medical emergencies. Symptoms of botulism include double or blurred vision, drooping eyelids, slurred speech, difficulty swallowing, dry mouth, and muscle weakness. Prompt medical care is necessary for treatment. To prevent botulism, it is important to be cautious when canning foods at home, avoid giving honey to babies, and seek medical care for infected wounds. In the United States, there are an average of 145 reported cases of botulism each year, with foodborne, infant, and wound botulism being the most common types.
Thought:{
    "action": "Final Answer"

{'input': 'Is this a serious illness?',
 'chat_history': [HumanMessage(content='I have a patient that can have Botulism, \nhow can I confirm the diagnose?'),
  AIMessage(content="To confirm a diagnosis of botulism, physicians may consider the patient's history and physical examination. However, these clues are usually not enough to confirm the diagnosis. Special tests may be needed to rule out other conditions with similar symptoms, such as Guillain-Barré syndrome, stroke, and myasthenia gravis. These tests may include a brain scan, spinal fluid examination, nerve conduction test, and a tensilon test for myasthenia gravis. Additionally, tests for botulinum toxin and the bacteria that cause botulism can be performed at certain laboratories.")],
 'output': 'Yes, botulism is a serious illness that can be fatal if not promptly treated.'}

And the memory works perfectly. We can maintain a conversation, taking into account that the model knows the previous questions and answers.

# Conclusions.
El experimento ha sido un pequeño Exito. Se ha configurado la base de datos de Vectores se ha llenado con información del Dataset. Se ha creado un agente de Langchain, y este ha sido capaz de ir a buscar la información a la base de datos tan solo cuando ha sido necesario. Sin olvidar que nuestro ChatBot tiene memoria.

Todo esto en unas pocas lineas de código!


---